# Arquitetura de Agente Autônomo para Navegação em Grafo (Livro-Jogo)

Este notebook implementa um agente autônomo projetado para navegar em uma narrativa de RPG estruturada como um livro-jogo. A estrutura do livro-jogo funciona como um **grafo direcionado**, onde cada página é um **nó** e as escolhas são as **arestas** que conectam os nós.

O objetivo principal é testar e validar a capacidade do agente de:
1.  **Observar** o estado atual (o texto da página).
2.  **Orientar-se** com base em suas instruções e na sua "ficha de personagem" (seu estado interno).
3.  **Decidir** qual a melhor ação (escolha) a tomar.
4.  **Agir** para transitar para o próximo estado (a próxima página).

Este ciclo é a implementação prática do modelo arquitetural **OODA (Observe, Orient, Decide, Act)**, um framework conceitual para tomada de decisão em cenários dinâmicos. Cada bloco de código abaixo representa um componente fundamental desta arquitetura.

### 1. Template da Ficha de Personagem

A função `create_character_sheet` é a fundação do estado do agente. Ela gera um dicionário Python que serve como um template em branco para a ficha de um personagem.

**Funcionamento:**
- **Estrutura Hierárquica**: A ficha é organizada em seções como `info`, `resources`, `skills`, e `status`, facilitando o acesso e a modificação de atributos específicos.
- **Valores Padrão**: Todos os campos são inicializados com valores padrão (zeros, listas vazias ou `None`). Isso garante que o agente sempre tenha uma estrutura de dados consistente, mesmo antes da personalização.
- **Pronta para Expansão**: A estrutura inclui seções como `modifiers` e `expert` skills, que são projetadas para serem preenchidas dinamicamente durante o jogo, à medida que o personagem ganha novas habilidades ou sofre penalidades.

In [1]:
import random

def create_character_sheet():
    """Cria um template para a ficha de personagem."""
    return {
        "info": {
            "name": "Character Name",
            "occupation": None,
            "age": 30,
            "backstory": ""
        },
        "contacts": {},
        "case_files": [],
        "magic": {"spells": [], "signare": []},
        "characteristics": {
            "STR": {"full": 0, "half": 0}, "CON": {"full": 0, "half": 0},
            "DEX": {"full": 0, "half": 0}, "INT": {"full": 0, "half": 0},
            "POW": {"full": 0, "half": 0}
        },
        "resources": {
            "luck": {"starting": 0, "current": 0},
            "magic_pts": {"starting": 0, "current": 0},
            "mov": 8
        },
        "skills": {
            "common": {
                "Athletics": {"full": 30, "half": 15}, "Drive": {"full": 30, "half": 15},
                "Navigate": {"full": 30, "half": 15}, "Observation": {"full": 30, "half": 15},
                "Read Person": {"full": 30, "half": 15}, "Research": {"full": 30, "half": 15},
                "Social": {"full": 30, "half": 15}, "Stealth": {"full": 30, "half": 15},
            },
            "combat": {
                "Fighting": {"full": 30, "half": 15}, "Firearms": {"full": 30, "half": 15}
            },
            "expert": {}
        },
        "status": {
            "damage_levels": ["Healthy", "Hurt", "Bloodied", "Down", "Impaired"], #0 for healthy, 1 for Hurt, 2 for Bloodied, 3 for Down, 4 for Impaired
            "damage_taken": 0,
            "modifiers": []  # e.g., {"skill": "Fighting", "type": "penalty_dice", "duration": "scene"}
        },
        "inventory": {"equipment": [], "weapons": []},
        "page_history": []
    }

### 2. Configuração Inicial do Personagem

A função `setup_character` personaliza a ficha de personagem criada anteriormente. Ela recebe a ficha em branco e os dados básicos do personagem (nome, ocupação) e a preenche com os valores iniciais corretos.

**Funcionamento:**
- **Dados Básicos**: Preenche as informações de identidade do personagem.
- **Recursos Iniciais**: Calcula e define recursos iniciais, como a Sorte (`luck`), usando uma rolagem de dados para adicionar variabilidade.
- **Perícias de Ocupação**: A lógica principal está em ajustar as perícias (`skills`) com base na `occupation` escolhida. Isso simula a especialização de uma classe de personagem em um RPG, dando bônus em perícias relevantes (ex: um `Police Officer` tem `Law` e `Fighting` aprimorados).

In [2]:
def setup_character(sheet, name, occupation, backstory):
    """Configura a ficha de personagem com base na ocupação e história."""
    sheet["info"]["name"] = name
    sheet["info"]["occupation"] = occupation
    sheet["info"]["backstory"] = backstory

    # Define a sorte inicial do personagem
    luck_roll = random.randint(1, 10) + random.randint(1, 10) + 50
    sheet["resources"]["luck"]["starting"] = luck_roll
    sheet["resources"]["luck"]["current"] = luck_roll

    # Ajusta as perícias com base na ocupação
    if occupation == "Police Officer":
        for skill in ["Law", "Social", "Athletics", "Fighting"]:
            if skill in sheet["skills"]["common"]:
                sheet["skills"]["common"][skill] = {"full": 60, "half": 30}
        sheet["skills"]["expert"]["Magic"] = {"full": 60, "half": 30}
        sheet["skills"]["expert"]["Law"] = {"full": 60, "half": 30}
    elif occupation == "Social Worker":
        for skill in ["Observation", "Research", "Social"]:
            if skill in sheet["skills"]["common"]:
                sheet["skills"]["common"][skill] = {"full": 60, "half": 30}
        sheet["skills"]["expert"]["Magic"] = {"full": 60, "half": 30}
    elif occupation == "Nurse":
        for skill in ["Observation", "Read Person", "Social"]:
            if skill in sheet["skills"]["common"]:
                sheet["skills"]["common"][skill] = {"full": 60, "half": 30}
        sheet["skills"]["expert"]["Medicine"] = {"full": 60, "half": 30}
        sheet["skills"]["expert"]["Magic"] = {"full": 60, "half": 30}

    return sheet

### 3. Mecânica de Rolagem de Dados (D100)

A função `make_check` é o motor de resolução de ações do RPG. Ela simula uma rolagem de dado de 100 lados (D100) e determina o nível de sucesso de uma ação com base no valor de uma perícia.

**Funcionamento:**
- **Rolagem D100**: Simula uma rolagem de percentil combinando um dado de dezenas e um de unidades.
- **Bônus e Penalidade**: Implementa a mecânica de "vantagem" e "desvantagem". A função aceita os parâmetros `bonus_dice` e `penalty_dice`. Com um dado de bônus, o menor de dois dados de dezena é usado. Com um dado de penalidade (aplicado pelo `perform_action` se houver um modificador ativo), o maior é usado. Se ambos estiverem presentes, eles se anulam.
- **Níveis de Sucesso**: O resultado da rolagem é comparado com o valor da perícia (`target_value`) e sua metade (`half_value`) para determinar um de cinco níveis de sucesso, retornados como um valor numérico:
    - **5**: Sucesso Crítico (rolagem 1)
    - **4**: Sucesso Difícil (rolagem <= metade da perícia)
    - **3**: Sucesso Normal (rolagem <= perícia)
    - **2**: Falha (rolagem > perícia)
    - **1**: Falha Crítica (rolagem 100)
- **Saída Declarativa**: O retorno numérico (`level`) permite que o sistema encontre a consequência correta de forma declarativa, sem precisar de lógica complexa no código que chama a função.

In [3]:
def make_check(target_value, half_value, bonus_dice=False, penalty_dice=False):
    """Realiza um teste de perícia D100 e retorna o nível de sucesso numérico."""
    tens_roll_1 = random.randint(0, 9) * 10
    tens_roll_2 = random.randint(0, 9) * 10
    units_roll = random.randint(1, 10)

    # Um dado de bônus e um dado de penalidade se anulam.
    if bonus_dice and penalty_dice:
        bonus_dice = False
        penalty_dice = False

    if bonus_dice:
        final_tens = min(tens_roll_1, tens_roll_2)
        print("Applied bonus die.")
    elif penalty_dice:
        final_tens = max(tens_roll_1, tens_roll_2)
        print("Applied penalty die.")
    else:
        final_tens = tens_roll_1

    # Calcula o resultado final da rolagem
    if final_tens == 0 and units_roll == 10:
        final_roll = 100
    elif final_tens == 0:
        final_roll = units_roll
    else:
        final_roll = final_tens + (units_roll % 10)

    # Determina o nível de sucesso
    if final_roll == 1:
        return (5, final_roll)  # Critical Success
    if final_roll == 100:
        return (1, final_roll)  # Fumble
    if final_roll <= half_value:
        return (4, final_roll)  # Hard Success
    if final_roll <= target_value:
        return (3, final_roll)  # Success
    
    return (2, final_roll)  # Failure

### 4. A Classe `Agent`: O Coração do Sistema

A classe `Agent` encapsula toda a lógica de estado, decisão e ação, funcionando como a "espinha dorsal" do jogador autônomo. A seguir, seus métodos serão detalhados um a um.

#### 4.1. Construtor (`__init__`) e Representação (`__repr__`)

**`__init__(self, name, occupation, game_instructions, game_data)`**
- **Propósito**: Inicializar o agente e seu estado interno.
- **Funcionamento**:
    1.  Chama `create_character_sheet()` para obter um template de ficha de personagem.
    2.  Chama `setup_character()` para personalizar a ficha com os dados do personagem (nome, ocupação, etc.).
    3.  Armazena as dependências injetadas (`game_data`, `game_instructions`) como atributos da instância.
    4.  Define o estado inicial do jogo, como a `current_page` (página inicial) e o `combat_status`.

**`__repr__(self)`**
- **Propósito**: Fornecer uma representação textual clara e concisa do objeto `Agent`.
- **Funcionamento**: Retorna uma string que mostra o nome e a ocupação do agente, útil para logs e debugging.

In [4]:
class Agent:
    """
    Implementa um agente autônomo que navega por um livro-jogo
    seguindo o ciclo OODA (Observe, Orient, Decide, Act).
    """
    
    def __init__(self, name, occupation, game_instructions, game_data):
        """
        Inicializa o agente, criando sua ficha de personagem e configurando o estado inicial.
        """
        base_sheet = create_character_sheet()
        self.sheet = setup_character(base_sheet, name, occupation, game_instructions.get_backstory())
        self.game_data = game_data
        self.current_page = 1
        self.combat_status = {}

    def __repr__(self):
        """Retorna uma representação textual do agente."""
        return f"Agent(Name: {self.sheet['info']['name']}, Occupation: {self.sheet['info']['occupation']})"

#### 4.2. O Método de Decisão (`_llm_decide`)

**`_llm_decide(self, choices)`**
- **Propósito**: Simular o processo de "Decisão" do ciclo OODA. Este método avalia as opções disponíveis (`choices`) e seleciona a mais apropriada com base no estado atual do agente e em sua lógica interna.
- **Funcionamento**:
    1.  **Itera sobre as Opções**: Percorre cada `choice` fornecida na página atual do livro-jogo.
    2.  **Verifica Pré-requisitos (`requires`)**: Para cada opção, verifica se existe um campo `requires`. Este campo define as condições necessárias para que a opção seja válida.
        - **Condições de Atributo**: Pode checar o valor de um atributo na ficha do personagem (ex: `"occupation": "Police Officer"`).
        - **Condições de Estado**: Pode checar o estado do jogo (ex: dano sofrido, `"damage_taken": {"min": 1}`).
    3.  **Lógica de Seleção**:
        - Se uma opção não tem `requires`, ela é considerada uma "opção padrão" ou "catch-all" e é armazenada como fallback.
        - Se uma opção tem `requires` e todas as condições são atendidas pelo estado atual do agente, essa opção é escolhida imediatamente e o método retorna.
    4.  **Retorno**: Se nenhuma opção com pré-requisitos for satisfeita, o método retorna a opção padrão que foi armazenada. Isso garante que o agente sempre tome uma decisão.
- **Design Declarativo**: A refatoração para usar o campo `requires` tornou a lógica de decisão puramente declarativa. O método não precisa mais interpretar strings de texto; ele apenas compara dados estruturados, tornando o sistema mais robusto e fácil de manter.

In [5]:
def _validate_choices(self, choices):
    """Valida se a lista de choices está em formato correto."""
    if not isinstance(choices, list):
        print(f"ERRO CRÍTICO: 'choices' deve ser uma lista, recebido: {type(choices)}")
        return False
    
    if len(choices) == 0:
        print("ERRO: Lista de choices está vazia.")
        return False
    
    for i, choice in enumerate(choices):
        if not isinstance(choice, dict):
            print(f"ERRO: Choice {i} deve ser um dicionário, recebido: {type(choice)}")
            return False
    
    return True

def _create_fallback_choice(self):
    """Cria uma choice de fallback segura quando todas as outras falham."""
    return {
        "text": "Ação de segurança - manter posição atual",
        "goto": max(1, self.current_page),  # Ficar na página atual ou ir para página 1
        "outcome": "Aguardando próxima oportunidade de ação."
    }

def _llm_decide(self, choices):
    """
    Decide qual ação tomar com base nas opções e no estado do agente.
    Esta versão usa uma lógica puramente declarativa baseada no campo 'requires'
    e também trata escolhas condicionais baseadas na ocupação.
    Inclui validações robustas para prevenir problemas com dados corrompidos.
    """
    # VALIDAÇÃO CRÍTICA: Verificar se choices está em formato válido
    if not self._validate_choices(choices):
        print("ERRO CRÍTICO: Lista de choices inválida. Usando ação de fallback.")
        return self._create_fallback_choice()
    
    default_choice = None
    
    try:
        for choice in choices:
            # Validação individual da choice
            if not isinstance(choice, dict):
                print(f"AVISO: Choice inválida (não é dicionário): {choice}. Pulando.")
                continue
            
            # Trata escolhas condicionais (estrutura especial com conditional_on)
            if "conditional_on" in choice:
                if choice["conditional_on"] == "occupation":
                    occupation = self.sheet["info"]["occupation"]
                    paths = choice.get("paths")
                    
                    if not isinstance(paths, dict):
                        print(f"ERRO: 'paths' deve ser um dicionário: {paths}. Pulando choice condicional.")
                        continue
                    
                    # Verifica se há um caminho específico para a ocupação atual
                    if occupation in paths:
                        selected_path = paths[occupation]
                        if isinstance(selected_path, dict):
                            print(f"Agente decidiu com base na ocupação ({occupation}): {selected_path}")
                            return selected_path
                        else:
                            print(f"ERRO: Caminho para ocupação deve ser um dicionário: {selected_path}")
                    # Usa o caminho padrão se não houver específico
                    elif "default" in paths:
                        selected_path = paths["default"]
                        if isinstance(selected_path, dict):
                            print(f"Agente decidiu pelo caminho padrão da ocupação: {selected_path}")
                            return selected_path
                        else:
                            print(f"ERRO: Caminho padrão deve ser um dicionário: {selected_path}")
                continue
            
            # Se a escolha não tiver pré-requisitos, é candidata a padrão
            if "requires" not in choice:
                if not default_choice:  # Armazena a primeira padrão encontrada
                    # Validar se a choice padrão tem campos mínimos necessários
                    if any(field in choice for field in ["goto", "roll", "opposed_roll", "luck_roll"]):
                        default_choice = choice
                continue
                
            # Avalia as condições em 'requires'
            requires = choice.get("requires")
            if not isinstance(requires, dict):
                print(f"AVISO: 'requires' deve ser um dicionário: {requires}. Pulando choice.")
                continue
                
            conditions_met = True
            for key, value in requires.items():
                try:
                    # Condição de ocupação
                    if key == "occupation":
                        if not isinstance(value, str):
                            print(f"AVISO: Valor de ocupação deve ser string: {value}")
                            conditions_met = False
                            break
                        if self.sheet["info"]["occupation"] != value:
                            conditions_met = False
                            break
                    # Condição de dano
                    elif key == "damage_taken":
                        if not isinstance(value, dict):
                            print(f"AVISO: Condição damage_taken deve ser dicionário: {value}")
                            conditions_met = False
                            break
                        min_damage = value.get("min", 0)
                        max_damage = value.get("max", float('inf'))
                        if not isinstance(min_damage, (int, float)) or not isinstance(max_damage, (int, float)):
                            print(f"AVISO: Valores min/max de damage_taken devem ser numéricos")
                            conditions_met = False
                            break
                        current_damage = self.sheet["status"]["damage_taken"]
                        if not (min_damage <= current_damage <= max_damage):
                            conditions_met = False
                            break
                    # Adicionar outras verificações de condição aqui, se necessário
                    else:
                        print(f"AVISO: Condição desconhecida '{key}' em requires. Ignorando.")
                except Exception as e:
                    print(f"ERRO ao avaliar condição '{key}': {e}")
                    conditions_met = False
                    break
            
            # Se todas as condições forem atendidas, escolhe esta opção
            if conditions_met:
                # Validar se a choice tem campos necessários antes de retornar
                if any(field in choice for field in ["goto", "roll", "opposed_roll", "luck_roll"]):
                    print(f"Agente decidiu com base em pré-requisitos: {choice}")
                    return choice
                else:
                    print(f"AVISO: Choice com pré-requisitos atendidos não tem ação válida: {choice}")
        
        # Se nenhuma escolha com pré-requisitos foi satisfeita, usa a padrão
        if default_choice:
            print(f"Agente decidiu pela opção padrão: {default_choice}")
            return default_choice
        
        # Se não há escolha padrão válida, pega a primeira choice da lista que tenha ação válida
        for choice in choices:
            if isinstance(choice, dict) and any(field in choice for field in ["goto", "roll", "opposed_roll", "luck_roll"]):
                print(f"Agente usando primeira choice válida como fallback: {choice}")
                return choice
    
    except Exception as e:
        print(f"ERRO CRÍTICO durante decisão: {e}")
    
    # Fallback de segurança final
    print("ERRO: Nenhuma choice válida encontrada. Usando fallback de segurança.")
    return self._create_fallback_choice()

Agent._validate_choices = _validate_choices
Agent._create_fallback_choice = _create_fallback_choice
Agent._llm_decide = _llm_decide

#### 4.3. O Método de Ação (`perform_action`)

**`perform_action(self, choice)`**
- **Propósito**: Executar a "Ação" do ciclo OODA. Este método pega a `choice` selecionada pelo `_llm_decide` e aplica seus efeitos no estado do agente e do jogo.
- **Funcionamento**:
    1.  **Processa Efeitos**: Verifica se a `choice` ou o `result` da rolagem têm um campo `effects`. Se tiver, itera sobre a lista de efeitos e chama o método correspondente (`take_damage`, `apply_penalty`, `heal_status`, etc.). Isso torna o sistema de consequências modular e expansível.
    2.  **Executa Rolagens de Dados**:
        - Se a `choice` contém um campo `roll`, significa que uma rolagem de perícia é necessária. O método identifica a perícia, busca o valor correspondente na ficha do personagem e chama `make_check()` para obter o nível de sucesso.
        - O resultado da rolagem (o `level`) é usado como chave para encontrar a consequência correta no dicionário `results` da `choice`. O `outcome` (texto descritivo), os `effects` e o `goto` são extraídos do resultado.
    3.  **Navegação Direta**: Se a `choice` não exige uma rolagem, ela provavelmente tem um `goto` direto, que é usado para navegar para a próxima página.
- **Validações Implementadas**: 
    - **Validação de Estrutura**: Verifica se `choice` é um dicionário válido com campos de ação apropriados
    - **Validação de Tipos**: Confirma que campos como `goto`, `roll`, `results` estão nos tipos corretos
    - **Validação de Valores**: Garante que páginas sejam números positivos, perícias sejam strings válidas, etc.
    - **Tratamento de Erros**: Captura exceções e fornece ações de fallback seguras
    - **Logs Detalhados**: Registra avisos e erros para debugging
- **Retorno**: O método retorna o `outcome` (a descrição do que aconteceu) para que o loop principal do jogo possa exibi-lo. Em caso de erro crítico, retorna uma mensagem de erro descritiva.

In [6]:
def _process_effects(self, effects):
    """Processa uma lista de efeitos no estado do agente."""
    if not isinstance(effects, list):
        print(f"AVISO: 'effects' deve ser uma lista, recebido: {type(effects)}. Ignorando efeitos.")
        return
        
    for effect in effects:
        if not isinstance(effect, dict):
            print(f"AVISO: Efeito deve ser um dicionário, recebido: {type(effect)}. Pulando efeito.")
            continue
            
        action = effect.get("action")
        if not action:
            print(f"AVISO: Efeito sem campo 'action': {effect}. Pulando efeito.")
            continue
            
        if action == "spend_luck":
            amount = effect.get("amount", 0)
            if not isinstance(amount, (int, float)) or amount < 0:
                print(f"AVISO: Quantidade de sorte inválida: {amount}. Usando 0.")
                amount = 0
            luck_amount = self.sheet["resources"]["luck"]["current"]
            luck_amount = max(0, luck_amount - amount)
            self.sheet["resources"]["luck"]["current"] = luck_amount
            print(f"Spent {amount} luck. Current luck: {self.sheet['resources']['luck']['current']}")
        elif action == "gain_skill":
            skill = effect.get("skill")
            if not skill or not isinstance(skill, str):
                print(f"AVISO: Nome de perícia inválido: {skill}. Pulando efeito.")
                continue
            if skill not in self.sheet["skills"]["common"]:
                self.sheet["skills"]["common"][skill] = {"full": 60, "half": 30}
                print(f"Gained skill {skill}. Current level: {self.sheet['skills']['common'][skill]}")
        elif action == "spend_magic":
            amount = effect.get("amount", 0)
            if not isinstance(amount, (int, float)) or amount < 0:
                print(f"AVISO: Quantidade de magia inválida: {amount}. Usando 0.")
                amount = 0
            magic_amount = self.sheet["resources"]["magic_pts"]["current"]
            magic_amount = max(0, magic_amount - amount)
            self.sheet["resources"]["magic_pts"]["current"] = magic_amount
            print(f"Spent {amount} magic points. Current magic points: {self.sheet['resources']['magic_pts']['current']}")
        elif action == "apply_penalty":
            self.apply_penalty(effect)
        elif action == "heal_damage":
            self.heal_status(effect)
        elif action == "take_damage":
            self.take_damage(effect)
        else:
            print(f"AVISO: Ação desconhecida '{action}' em efeito: {effect}. Pulando efeito.")

def _validate_choice(self, choice):
    """Valida se a choice retornada pelo LLM está em formato correto."""
    if not isinstance(choice, dict):
        print(f"ERRO CRÍTICO: Choice deve ser um dicionário, recebido: {type(choice)}")
        return False
    
    # Verifica se tem pelo menos um campo válido para ação
    valid_action_fields = ["goto", "roll", "opposed_roll", "luck_roll", "effects"]
    has_valid_action = any(field in choice for field in valid_action_fields)
    
    if not has_valid_action:
        print(f"ERRO: Choice não contém nenhum campo de ação válido: {choice}")
        return False
    
    # Validações específicas por tipo de ação
    if "goto" in choice:
        goto_value = choice["goto"]
        if not isinstance(goto_value, int) or goto_value < 0:
            print(f"ERRO: 'goto' deve ser um número inteiro positivo, recebido: {goto_value}")
            return False
    
    if "roll" in choice:
        roll_value = choice["roll"]
        if not isinstance(roll_value, (str, dict)):
            print(f"ERRO: 'roll' deve ser string ou dicionário, recebido: {type(roll_value)}")
            return False
        
        if isinstance(roll_value, dict) and "skill" not in roll_value:
            print(f"ERRO: 'roll' como dicionário deve ter campo 'skill': {roll_value}")
            return False
    
    if "results" in choice:
        results = choice["results"]
        if not isinstance(results, dict):
            print(f"ERRO: 'results' deve ser um dicionário, recebido: {type(results)}")
            return False
    
    return True

def perform_action(self, choice):
    """
    Executa a ação decidida, aplicando efeitos e rolagens de dados.
    Inclui validações para prevenir problemas com respostas incorretas do LLM.
    """
    # VALIDAÇÃO CRÍTICA: Verificar se choice está em formato válido
    if not self._validate_choice(choice):
        print("ERRO CRÍTICO: Choice inválida recebida do LLM. Usando ação padrão de segurança.")
        # Ação de segurança: tentar navegar para página 1 ou manter página atual
        self.current_page = max(1, self.current_page)
        return "Ação de segurança executada devido a choice inválida."
    
    outcome = choice.get("outcome", "")
    
    try:
        # 1. Aplicar efeitos imediatos da escolha
        if "effects" in choice:
            self._process_effects(choice["effects"])

        # 2. Executar rolagens de dados, se necessário
        if "roll" in choice:        
            roll_data = choice["roll"]
            
            # Detectar se roll é uma string (formato simples) ou dicionário (formato complexo)
            if isinstance(roll_data, str):
                # Formato simples: "roll": "Fighting"
                skill_name = roll_data
                difficulty = choice.get("difficulty", "normal")
                bonus_dice = choice.get("bonus_dice", False)
                penalty_dice = choice.get("penalty_dice", False)
                results = choice.get("results", {})
            elif isinstance(roll_data, dict):
                # Formato complexo: "roll": {"skill": "INT", "difficulty": "hard", ...}
                skill_name = roll_data.get("skill")
                difficulty = roll_data.get("difficulty", "normal")
                bonus_dice = roll_data.get("bonus_dice", False)
                penalty_dice = roll_data.get("penalty_dice", False)
                results = roll_data.get("results", {})
            else:
                print(f"ERRO: Formato de 'roll' inválido: {roll_data}. Usando valores padrão.")
                skill_name = "Athletics"  # Skill padrão segura
                difficulty = "normal"
                bonus_dice = False
                penalty_dice = False
                results = {}

            # Validar skill_name
            if not skill_name or not isinstance(skill_name, str):
                print(f"ERRO: Nome de perícia inválido: {skill_name}. Usando 'Athletics'.")
                skill_name = "Athletics"

            # Encontra a perícia na ficha do personagem
            skill_values = None
            if skill_name in self.sheet["skills"]["common"]:
                skill_values = self.sheet["skills"]["common"][skill_name]
            elif skill_name in self.sheet["skills"]["combat"]:
                skill_values = self.sheet["skills"]["combat"][skill_name]
            elif skill_name in self.sheet["skills"]["expert"]:
                skill_values = self.sheet["skills"]["expert"][skill_name]
            else:
                # Para características como INT, DEX, etc.
                if skill_name in self.sheet["characteristics"]:
                    char_value = self.sheet["characteristics"][skill_name]["full"]
                    skill_values = {"full": char_value, "half": char_value // 2}
                else:
                    print(f"Perícia '{skill_name}' não encontrada. Usando valores padrão.")
                    skill_values = {"full": 30, "half": 15}

            # Validar difficulty
            if difficulty not in ["normal", "hard"]:
                print(f"AVISO: Dificuldade inválida '{difficulty}'. Usando 'normal'.")
                difficulty = "normal"

            # Ajusta valores baseado na dificuldade
            if difficulty == "hard":
                target_value = skill_values["half"]
                half_value = target_value // 2
            else:  # normal
                target_value = skill_values["full"]
                half_value = skill_values["half"]
            
            # Validar bonus_dice e penalty_dice
            bonus_dice = bool(bonus_dice) if isinstance(bonus_dice, (bool, int)) else False
            penalty_dice = bool(penalty_dice) if isinstance(penalty_dice, (bool, int)) else False

            # Verifica se há penalidades ativas para esta perícia
            has_penalty = any(
                mod.get("skill") == skill_name and mod.get("type") == "penalty_dice" 
                for mod in self.sheet["status"]["modifiers"] if isinstance(mod, dict)
            )

            has_bonus = any(
                mod.get("skill") == skill_name and mod.get("type") == "bonus_dice" 
                for mod in self.sheet["status"]["modifiers"] if isinstance(mod, dict)
            )

            # Realiza o teste
            level, roll_value = make_check(target_value, half_value, bonus_dice=has_bonus, penalty_dice=has_penalty)
            print(f"Rolled {skill_name}: {roll_value} vs {target_value} -> Level {level}")

            # Processa os resultados baseados no nível de sucesso
            if not isinstance(results, dict):
                print(f"ERRO: 'results' deve ser um dicionário: {results}")
                results = {}
            
            result = results.get(str(level))
            
            if result:
                # Para formato complexo, result pode ser um número (goto direto) ou dict com outcome/effects/goto
                if isinstance(result, int):
                    # Resultado simples: apenas o número da página
                    if result > 0:  # Validar página válida
                        self.current_page = result
                        print(f"Navigating to page {result} based on roll result.")
                    else:
                        print(f"ERRO: Página inválida {result}. Mantendo página atual.")
                elif isinstance(result, dict):
                    # Resultado complexo: contém outcome, effects, goto
                    outcome = result.get("outcome", outcome)
                    if "effects" in result:
                        self._process_effects(result["effects"])
                    if "goto" in result:
                        goto_page = result["goto"]
                        if isinstance(goto_page, int) and goto_page > 0:
                            self.current_page = goto_page
                            print(f"Navigating to page {goto_page} based on roll result.")
                        else:
                            print(f"ERRO: Página 'goto' inválida: {goto_page}. Mantendo página atual.")
            else:
                print(f"AVISO: Nenhum resultado encontrado para nível {level}. Mantendo página atual.")
        
        # 2.5. Executar rolagens de sorte (luck_roll)
        elif "luck_roll" in choice and choice["luck_roll"]:
            # Usa o valor atual de sorte do personagem
            luck_value = self.sheet["resources"]["luck"]["current"]
            luck_half = luck_value // 2
            
            # Realiza o teste de sorte
            level, roll_value = make_check(luck_value, luck_half)
            
            # Processa o resultado da rolagem de sorte
            results = choice.get("results", {})
            if not isinstance(results, dict):
                print(f"ERRO: 'results' para luck_roll deve ser um dicionário: {results}")
                return outcome
                
            result = results.get(str(level))
            if not result: # Fallback para o nível de sucesso mais próximo
                 for i in range(level - 1, 0, -1):
                    result = results.get(str(i))
                    if result:
                        break
            
            print(f"Rolled Luck: {roll_value} vs {luck_value} -> Level {level}")
            
            if result:
                outcome = result.get("outcome", outcome)
                if "effects" in result:
                    self._process_effects(result["effects"])
                if "goto" in result:
                    goto_page = result["goto"]
                    if isinstance(goto_page, int) and goto_page > 0:
                        self.current_page = goto_page
                        print(f"Navigating to page {goto_page} based on luck roll.")
                    else:
                        print(f"ERRO: Página 'goto' inválida: {goto_page}. Mantendo página atual.")

        # 3. Opposed roll
        elif "opposed_roll" in choice:
            skill_to_roll = choice["opposed_roll"]
            if not isinstance(skill_to_roll, str):
                print(f"ERRO: 'opposed_roll' deve ser uma string: {skill_to_roll}")
                return outcome
                
            opponent_skill = choice.get("opponent_skill", {"full": 30, "half": 15})
            if not isinstance(opponent_skill, dict):
                print(f"ERRO: 'opponent_skill' deve ser um dicionário: {opponent_skill}")
                opponent_skill = {"full": 30, "half": 15}
                
            results = choice.get("outcomes", {})
            if not isinstance(results, dict):
                print(f"ERRO: 'outcomes' deve ser um dicionário: {results}")
                return outcome

            # Encontra a perícia na ficha do personagem
            skill_values = None
            if skill_to_roll in self.sheet["skills"]["common"]:
                skill_values = self.sheet["skills"]["common"][skill_to_roll]
            elif skill_to_roll in self.sheet["skills"]["combat"]:
                skill_values = self.sheet["skills"]["combat"][skill_to_roll]
            elif skill_to_roll in self.sheet["skills"]["expert"]:
                skill_values = self.sheet["skills"]["expert"][skill_to_roll]
            else:
                # Para características como INT, DEX, etc.
                if skill_to_roll in self.sheet["characteristics"]:
                    char_value = self.sheet["characteristics"][skill_to_roll]["full"]
                    skill_values = {"full": char_value, "half": char_value // 2}
                else:
                    print(f"Perícia '{skill_to_roll}' não encontrada. Usando valores padrão.")
                    skill_values = {"full": 30, "half": 15}

            target_value = skill_values["full"]
            half_value = skill_values["half"]

            # Verifica se há modificadores ativos para esta perícia
            has_penalty = any(
                mod.get("skill") == skill_to_roll and mod.get("type") == "penalty_dice"
                for mod in self.sheet["status"]["modifiers"] if isinstance(mod, dict)
            )

            has_bonus = any(
                mod.get("skill") == skill_to_roll and mod.get("type") == "bonus_dice"
                for mod in self.sheet["status"]["modifiers"] if isinstance(mod, dict)
            )

            # Realiza o teste do agente
            agent_level, agent_roll = make_check(target_value, half_value, bonus_dice=has_bonus, penalty_dice=has_penalty)
            print(f"Agente Rolled {skill_to_roll}: {agent_roll} vs {target_value} -> Level {agent_level}")

            # Realiza o teste do oponente
            opponent_level, opponent_roll = make_check(opponent_skill["full"], opponent_skill["half"])
            print(f"Oponente Rolled: {opponent_roll} vs {opponent_skill['full']} -> Level {opponent_level}")

            # Determina o resultado do confronto
            if agent_level > opponent_level:
                result_key = "win"
            elif agent_level < opponent_level:
                result_key = "lose"
            else:
                result_key = "draw"
            
            if result_key in results:
                result = results[result_key]
                if isinstance(result, dict):
                    if "effects" in result:
                        self._process_effects(result["effects"])
                    if "goto" in result and isinstance(result["goto"], int) and result["goto"] > 0:
                        self.current_page = result["goto"]
                        print(f"{result_key.upper()}: Navigating to page {result['goto']}.")
                    outcome = result.get("outcome", outcome)
                else:
                    print(f"ERRO: Resultado '{result_key}' deve ser um dicionário: {result}")

        # 4. Navegação direta (sem rolagem)
        elif "goto" in choice:
            goto_page = choice["goto"]
            if isinstance(goto_page, int) and goto_page > 0:
                print(f"Navigating directly to page {goto_page}.")
                self.current_page = goto_page
            else:
                print(f"ERRO: Página 'goto' inválida: {goto_page}. Mantendo página atual.")
        
        else:
            print(f"AVISO: Nenhuma ação reconhecida na choice: {choice}")
            
    except Exception as e:
        print(f"ERRO CRÍTICO durante execução da ação: {e}")
        print(f"Choice problemática: {choice}")
        outcome = f"Erro durante execução: {str(e)}"
        
    return outcome

Agent._process_effects = _process_effects
Agent._validate_choice = _validate_choice
Agent.perform_action = perform_action

#### 4.4. O Loop Principal do Agente (`run`)

**`run(self)`**
- **Propósito**: Orquestrar o ciclo OODA completo, servindo como o ponto de entrada para a execução do agente no livro-jogo.
- **Funcionamento**:
    1.  **Loop Infinito (Controlado)**: O método opera dentro de um `while True`, que continua executando o ciclo do agente até que uma condição de término seja alcançada (uma página sem opções ou um `goto` para `0`).
    2.  **Observação (`_observe`)**: No início de cada ciclo, chama o método `_observe` (a ser detalhado a seguir) para obter o estado atual do jogo (o texto e as opções da página atual).
    3.  **Orientação (`_orient`)**: Em seguida, chama `_orient` para processar as informações observadas e prepará-las para a decisão.
    4.  **Decisão (`_llm_decide`)**: Passa as `choices` para o `_llm_decide`, que seleciona a melhor ação.
    5.  **Ação (`perform_action`)**: Executa a `choice` selecionada através do `perform_action` e armazena o `outcome`.
    6.  **Condição de Parada**: O loop é interrompido se a `current_page` for `0` ou se a página atual não tiver `choices`, indicando o fim de um arco da história.
- **Ciclo Contínuo**: Este método encapsula a autonomia do agente, permitindo que ele navegue pela história de forma independente, página por página, até encontrar um ponto final.

In [7]:
def run(self):
    """
    Executa o ciclo OODA principal para navegar pelo livro-jogo.
    """
    while True:
        # 1. Observe
        page_text, choices = self._observe()
        if not choices:
            print("Fim da história (nenhuma escolha encontrada).")
            break
        
        # 2. Orient
        self._orient(page_text)
        
        # 3. Decide
        chosen_action = self._llm_decide(choices)
        print(f"Agente escolheu: {chosen_action}")        
        # 4. Act
        outcome = self.perform_action(chosen_action)
        print(f"Resultado: {outcome}\n---")
        # Condição de parada
        if self.current_page == 0:
            print("Fim da história (goto: 0).")
            break

Agent.run = run

#### 4.5. Métodos Auxiliares: `_observe` e `_orient`

Estes dois métodos completam as fases restantes do ciclo OODA, focando na coleta e processamento de informações.

**`_observe(self)`**
- **Propósito**: Implementar a fase de "Observação". O método coleta todos os dados brutos da página atual do livro-jogo.
- **Funcionamento**:
    1.  Acessa o `self.game_data` usando a `self.current_page` como chave.
    2.  Extrai o texto da página (`page_text`) e a lista de `choices` disponíveis.
    3.  Imprime o número da página e seu texto para que o usuário (ou desenvolvedor) possa acompanhar a jornada do agente.
- **Retorno**: Retorna uma tupla contendo `(page_text, choices)`.

**`_orient(self, page_text)`**
- **Propósito**: Implementar a fase de "Orientação". O método processa as informações brutas observadas e atualiza o estado interno do agente.
- **Funcionamento**:
    1.  Adiciona a `current_page` ao histórico de páginas visitadas (`page_history`) na ficha do personagem. Isso cria um registro do caminho percorrido pelo agente.
    2.  O `page_text` é passado como argumento, permitindo futuras expansões onde o agente poderia, por exemplo, usar um LLM para extrair informações contextuais do texto da página e atualizar sua "compreensão" da situação. Atualmente, esta função é simples, mas está projetada para ser extensível.

In [8]:
def _observe(self):
    """
    Observa o ambiente, lendo o texto e as opções da página atual.
    """
    page_data = self.game_data.get(self.current_page, {})
    page_text = page_data.get("text", "Página não encontrada.")
    choices = page_data.get("choices", [])
    
    print(f"--- Página {self.current_page} ---\n{page_text}\n")
    print(f"Escolhas disponíveis: {choices}")
    return page_text, choices

def _orient(self, page_text):
    """
    Orienta o agente, atualizando seu estado interno com base nas observações.
    """
    # Adiciona a página atual ao histórico
    if self.current_page not in self.sheet["page_history"]:
        self.sheet["page_history"].append((self.current_page, page_text))
    
    # Futuramente, poderia usar um LLM para extrair contexto do page_text
    pass

Agent._observe = _observe
Agent._orient = _orient

#### 4.6. Métodos de Efeitos: `apply_penalty`, `heal_status` e `take_damage`

Estes métodos são responsáveis por alterar o estado do personagem em resposta a eventos do jogo.

**`apply_penalty(self, effect)`**
- **Propósito**: Aplicar uma penalidade a uma perícia do personagem.
- **Funcionamento**:
    1.  Extrai os detalhes da penalidade (`skill`, `duration`) do dicionário `effect`.
    2.  Cria um novo dicionário representando o modificador.
    3.  Adiciona o modificador à lista `modifiers` na ficha do personagem.

**`heal_status(self, effect)`**
- **Propósito**: Curar um status negativo ou dano.
- **Funcionamento**:
    1.  Verifica o `status` a ser curado.
    2.  Atualmente, implementado para `damage`, ele redefine `damage_taken` e `current_damage` para seus estados iniciais (0 e `None`).

In [9]:
def apply_penalty(self, effect):
    """Aplica uma penalidade a uma perícia."""
    skill = effect["skill"] # e.g., "Fighting"
    duration = effect["duration"] # e.g., 1 number of rolls    
    self.sheet["status"]["modifiers"].append((skill, "penalty_dice", duration))
    print(f"Applied penalty to {skill} for {duration}.")

def heal_status(self, effect):
    """Cura um status ou dano."""
    status_to_heal = effect["amount"]
    self.sheet["status"]["damage_taken"] = max(0, self.sheet["status"]["damage_taken"] - status_to_heal)
    damage_level_map = self.sheet["status"]["damage_levels"]
    current_level = damage_level_map[self.sheet["status"]["damage_taken"]]
    print(f"Healed damage: {status_to_heal}. Total damage: {self.sheet['status']['damage_taken']}. Status: {current_level}")
Agent.apply_penalty = apply_penalty
Agent.heal_status = heal_status

**`take_damage(self, effect)`**
- **Propósito**: Aplicar dano ao personagem.
- **Funcionamento**:
    1.  Incrementa o `damage_taken` na ficha do personagem com o `amount` do efeito.
    2.  Compara o dano total com os `damage_levels` para determinar o novo `current_damage` status (ex: "Hurt", "Bloodied").

In [10]:
def take_damage(self, effect):
    """Aplica dano ao personagem e atualiza o status."""
    amount = effect.get("amount", 0)
    self.sheet["status"]["damage_taken"] += amount
    
    # Atualiza o nível de dano atual
    damage_level_map = self.sheet["status"]["damage_levels"]
    current_level = damage_level_map[self.sheet["status"]["damage_taken"]]

    print(f"Took {amount} damage. Total damage: {self.sheet['status']['damage_taken']}. Status: {current_level}")
    #se o level for Impaired, o jogo termina
    if current_level == "Impaired":
        print("Character is Impaired. Game Over.")
        self.current_page = 999

Agent.take_damage = take_damage

### 5. Execução Principal: Configurando e Rodando o Agente

Esta é a célula final que une todas as peças. Ela é responsável por:

1.  **Definir as Instruções do Jogo**: Cria um dicionário `game_instructions` que contém a história de fundo (`backstory`) do personagem. Em um sistema mais complexo, isso poderia incluir regras gerais, objetivos de longo prazo, etc.
2.  **Definir os Dados do Jogo**: O dicionário `game_data` representa o "livro-jogo" em si. Cada chave é o número de uma página, e o valor contém o texto da página e as `choices` (opções) disponíveis. É aqui que a estrutura declarativa brilha, com cada `choice` contendo toda a lógica necessária para a sua resolução (`requires`, `roll`, `goto`, `results`).
3.  **Instanciar o Agente**: Cria uma instância da classe `Agent`, injetando as `game_instructions` e o `game_data`. O nome e a ocupação do agente são definidos aqui.
4.  **Executar o Agente**: Chama o método `agent.run()`, que inicia o ciclo OODA e faz o agente navegar autonomamente pela história definida no `game_data`.

In [11]:
# Encapsulamento dos Dados e Instruções do Jogo
import pages

class GameInstructions:
    """Fornece as diretrizes de alto nível para o agente."""
    def get_backstory(self):
        return "Um oficial recém-formado na divisão de crimes especiais, conhecido por sua abordagem metódica."

class GameData:
    """Carrega e fornece acesso aos dados do livro-jogo."""
    def __init__(self):
        self.pages = pages.PAGES
    
    def get(self, page_id, default=None):
        return self.pages.get(page_id, default)

# Ponto de Entrada Principal
if __name__ == "__main__":
    # 1. Instanciar os componentes do jogo
    game_instructions = GameInstructions()
    game_data = GameData()

    # 2. Instanciar o agente com as dependências
    agent = Agent(
        name="Alex", 
        occupation="Police Officer", 
        game_instructions=game_instructions, 
        game_data=game_data
    )
    
    # 3. Executar o agente
    agent.run()

--- Página 1 ---
The 20-mph (32 kph) speed limit makes it easy to read the doors as you drive your Ford Escort east on Prince of Wales Road. The house numbers count down along a mishmash of terraces, interrupted only by the pillared front of a former Methodist chapel, now a contemporary art centre with the obligatory cafe and gift shop. You find what you're looking for in the low hundreds and, by some miracle, there is a parking space opposite, just wide enough for your car. Safely wedged into the parking space, you take a moment to review the report. Verbal disagreements in the basement flat, three nights in a row. Two voices heard, getting louder each time. Culminating on the third night in an almighty crash that prompted a 999 (emergency services) call from the couple upstairs, the Romanian students next door, and a cyclist delivering curry who was startled enough to drop his bag over the railing, ending up with an unholy amalgam of tikka masala, korma, and saag aloo. The responding